In [1]:
from modules.data import Data
from modules.rag import resolve_context
from modules.utils import resumable, save_progress

from tqdm import tqdm

In [2]:
data = Data()
data_test = data.get_split("test")

2024-07-16 17:26:50.133 | INFO     | modules.data:load_data:18 - load dev data, len: 1000
2024-07-16 17:26:50.140 | INFO     | modules.data:load_data:18 - load test data, len: 1000
2024-07-16 17:26:50.176 | INFO     | modules.data:load_data:18 - load train data, len: 5000


In [3]:
context_template = """
产品名：{product_name}
条款：{clause}
"""

In [4]:
@resumable(progress_file="./cache/progress_test.json")
def process_data_with_rag(data, context_template, start_idx=0, progress_file=None):
    for idx, item in tqdm(
        enumerate(data[start_idx:], start=start_idx), initial=start_idx, total=len(data)
    ):
        product_name, raw_context, query = item["产品名"], item["条款"], item["问题"]
        if len(raw_context) > 500:
            retrieved_nodes = resolve_context(
                idx=idx, raw_context=raw_context, query=query
            )
            context = ""
            for node_with_score in retrieved_nodes:
                context += node_with_score.node.text
        else:
            context = raw_context
        data[idx]["context"] = context_template.format(
            product_name=product_name, clause=context
        )

        # Save progress after each item is processed
        if progress_file:
            save_progress(progress_file, data, ensure_ascii=False, indent=4)

    return data


# 使用装饰器的函数
data_test_new = process_data_with_rag(data=data_test, context_template=context_template)

Resuming from index 3


  0%|          | 3/1000 [00:00<?, ?it/s]2024-07-16 17:26:50.210 | INFO     | modules.rag:resolve_context:15 - idx: 3, query: 如果在上述情形或期间内发生保险事故，保险公司如何处理已交保费？, raw_context: 责任免除 因下列一个或者多个情形引起的保险事故，本公司不承担相应的保险责任：1)投保人对被保险人的故意杀害、故意伤害；2)被保险人故意犯罪或者抗拒依法采取的刑事强制措施；3）被保险人故意自伤、或自本合同成立或者本合同效力恢复之日起2年内自杀，但被保险人自杀时为无?事行为能力人的除外；4）被保险人患有遗传性疾病（?7.7），先天性畸形、变形或染色体异常（?7.8）（在本合同有效期内罹患本条款第7.18条“轻度疾病”释义中约定的第20项轻度疾病、第7.20条“重度疾病”释义中约定的第35、40、45、56、61、70、80、82、99、100、101项重度疾病、第7.21条“特定疾病”释义中约定的第10项特定疾病除外）；5）战争、军事冲突、暴乱或武装叛乱；6)核爆炸、核辐射或核污染。被保险人在下列期间内发生的保险事故，本公司不承担相应的保险责任：1)被保险人服用、吸?或注射毒品（?7.9）；2)被保险人酒后驾驶(?7.10）、无合法有效驾驶证驾驶（?7.11），或驾驶无合法有效行驶证（?7.12）的机动?（?7.13）；3)被保险人感染艾滋病病毒或患艾滋病（?7.14）（在本合同有效期内罹患本条款第7.20条“重度疾病”释义中约定的第31、81项重度疾病除外）。被保险人因上述情形或在上述期间内发生本合同约定的保险事故的，本合同效力终止，本公司向投保人退还本合同的现金价值（?7.15）。但因投保人对被保险人的故意杀害、故意伤害导致被保险人发生本合同约定的保险事故的，本公司向其他权利人退还本合同的现金价值，其他权利人按照被保险人（第一顺位）、被保险人继承人（第二顺位）的顺序确定。其他免责条款除本条款第2.4条“责任免除”外，本合同中还有一些免除本公司责任的条款，详?本条款第2.3条“保险责任”、第3.3条“犹豫期后解除合同的手续及?险”、第4.4条“合同效力的中止和恢复”、第6.7条“款项扣除”和第7条“名词释义”等部分中以黑体字加下划线

2024-07-16 17:26:50.670 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

  1%|          | 12/1000 [00:52<25:20,  1.54s/it]  2024-07-16 17:27:42.594 | INFO     | modules.rag:resolve_context:15 - idx: 12, query: 如果被保险人从事高风险运动而发生意外，保险公司是否承担赔偿责任？, raw_context: 救援类通用责任免除 因下列情形之一造成的运送或运返费用、直接或间接造成被保险人身故或住院的、直接或间接造成被保险人身故或发生的费用，保险人不承担给付保险金责任：（1）被保险人接受包括美容、整容、整形手术在内的任何医疗行为而造成的意外；（2）被保险人受酒精、毒品、管制药物的影响，但遵医嘱使用药物的情形不在此限；（3）任何生物、化学、原子能武器，原子能或核能装置所造成的爆炸、灼伤、污染或辐射；（4）被保险人故意犯罪或抗拒依法采取的刑事强制措施；（5）被保险人存在精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）；（6）被保险人从事高风险运动；（7）被保险人酒后驾驶、无有效驾驶证驾驶或驾驶无有效行驶证的机动交通工具；（8）被保险人感染艾滋病病毒或患艾滋病。
因下列情形之一造成的运送或运返费用、直接或间接造成被保险人身故或住院的、直接或间接造成被保险人身故或发生的费用，保险人不承担给付保险金责任： （1）被保险人接受包括美容、整容、整形手术在内的任何医疗行为而造成的意外； （2）被保险人受酒精、毒品、管制药物的影响，但遵医嘱使用药物的情形不在此限； （3）任何生物、化学、原子能武器，原子能或核能装置所造成的爆炸、灼伤、污染或辐射； （4）被保险人故意犯罪或抗拒依法采取的刑事强制措施； （5）被保险人存在精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）； （6）被保险人从事高?险运动； （7）被保险人酒后驾驶、无有效驾驶证驾驶或驾驶无有效行驶证的机动交通工具； （8）被保险人感染艾滋病病毒或患艾滋病。
2024-07-16 17:27:42.600 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


...

 99%|█████████▉| 992/1000 [1:04:56<00:09,  1.15s/it]2024-07-16 18:31:46.567 | INFO     | modules.rag:resolve_context:15 - idx: 992, query: 在儿科方面，哪些疾病或情况下的患者仍可以投保健康福·少儿重疾险？, raw_context: 例外事项: 一、下述情况住院、手术或长期服药，仍可投保本产品:1.呼吸科:上呼吸道感染、感冒且痊愈、鼻/咽炎、鼻窦炎、扁桃体炎、扁桃体切除术、腺样体手 术、急性支气管炎或急性肺炎(非重症);2.消化科:急性肠胃炎、胃肠功能紊乱、阑尾炎、胆囊炎、胆囊结石、胆囊息肉手术且病理良性;3.骨科:上/下肢骨折且已痊愈、意外住院不超5天且已痊愈(无后遗症或器官缺损);4.泌尿科:肾/输尿管/膀胱结石但无肾积水或肾功能损害、腹股沟疝、鞘膜积液、尿道炎、包皮手 术;5.皮肤科:痤疮、湿疹、皮炎、皮脂腺囊肿(粉瘤)手术、脂肪瘤切除且病理良性、婴幼儿黄疸且 治愈(无脑损害或后遗症);6、儿科:先天性多指或兔唇;手足口病且已痊愈，咽峡炎且已痊愈，疫苗反应且已痊愈。二、乙肝，符合以下情况时，仍可投保本产品: 1.有近1年内乙肝检查，乙肝小三阳(HBsAg、HBeAb、HBcAb三项阳性)或乙肝小二阳 (HBsAg、HBcAb两项阳性)，肝功能(ALT、AST)完全正常，且肝脏超声正常或仅提示为肝囊 肿、肝血管瘤、脂肪肝; 2.或:有近1年内乙肝检查，乙肝表面抗体(HBsAb)阳性，且肝脏超声正常或仅提示为肝囊肿、 肝血管瘤、脂肪肝。若您选择了投保人豁免责任，则投保人需满足以下健康告知。 请如实告知投保人是否存在以下情况:1.就医行为及保险情况: 过去2年内，因病做过手术或住院治疗超过5天、或因病遵医嘱需连续服药超过30天; 过去2年曾被保险公司拒保、延期、加费或者附加相关条件承保。2.目前或曾经患有下列疾病或症状: 肿瘤:恶性肿瘤(含原位癌)、白血病、脑或脊髓的肿瘤或占位;
2024-07-16 18:31:46.573 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

100%|█████████▉| 995/1000 [1:05:15<00:16,  3.27s/it]2024-07-16 18:32:05.378 | INFO     | modules.rag:resolve_context:15 - idx: 995, query: 在保险合同生效前发生的保险事故，保险公司是否承担赔偿责任？, raw_context: 保险费的支付 本合同保险费根据约定的投保年龄等情况确定，并在保单中载明。本合同的保费支付方式分为一次性支付和分期支付，由您和我们双方约定，并在保险单上载明。若您选择一次性支付保险费，您应当在本合同成立时一次性交清保险费，保险费交清前，本合同不成立，对于保险合同生效前发生的保险事故，我们不承担赔偿责任。如您选择分期支付保险费，需申请并经被保险人同意，并在保险单中载明分期支付的周期。如您未支付首期保费，保险合同不成立，我们不承担保险责任。在支付首期保费后，您应当在每个保险费约定的支付日缴纳其余各月对应期次的保险费。如您未按照保险合同约定的付款时间支付当期保费，允许在宽限期内补交保险费。除另有约定外，如被保险人在正常缴纳保费对应的保险期限内或宽限期内发生保险事故，我们按照合同约定赔偿保险金，但须投保人先行补缴剩余全年保险费，补交金额为保险合同约定的全年保险费总额扣除投保人已缴纳的保险费。如您未按照保险合同约定的付款时间足额支付当期保险费，且在保险合同约定的缴费宽限期内未足额补交当期保险费，保险期限在上一缴费周期结束之日起终止，对于保险期限终止后发生的保险事故，我们不承担保险责任。除另有约定外，缴费宽限期为25天。
2024-07-16 18:32:05.383 | INFO     | modules.rag:resolve_context:36 - #nodes: 3


Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [1:05:36<00:00,  3.95s/it]
